In [1]:
from scene_splitter.clip_scene_splitter import CLIPSceneSplitter
from tracking.detector import ObjectDetector
from tracking.tracker import DeepSortTracker
from utils.video_tools import save_scenes_ffmpeg
from utils.audio_tools import extract_audio_ffmpeg
from audio.audio_features import get_audio_activity, transcribe_audio_whisper
from sentence_transformers import SentenceTransformer, util
from insightface.app import FaceAnalysis
from sklearn.cluster import DBSCAN
from tqdm import tqdm
import numpy as np
import onnxruntime
import cv2
import re

/home/artniz/hackatons/kion/kionvenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(onnxruntime.get_device())

GPU


In [3]:
def analyze_video(video_path, scene_splitter, detector, tracker, every_n_frames=15):
    scenes = scene_splitter.detect_scenes(video_path, every_n_frames=every_n_frames)

    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    scene_data = []
    track_faces = {}

    for (start, end) in tqdm(scenes, desc="Analyzing scenes"):
        cap.set(cv2.CAP_PROP_POS_MSEC, start * 1000)
        frame_idx = int(start * fps)
        end_idx = int(end * fps)
        track_ids_in_scene = set()

        while frame_idx < end_idx:
            ret, frame = cap.read()
            if not ret:
                break

            if frame_idx % every_n_frames == 0:
                detections = detector.detect(frame)
                tracks = tracker.update(detections, frame)

                for t in tracks:
                    track_id = t['track_id']
                    x1, y1, x2, y2 = map(int, t['bbox'])

                    # Добавим в сцену
                    track_ids_in_scene.add(track_id)

                    # Сохраняем лицо
                    face_crop = frame[y1:y2, x1:x2]
                    if face_crop.size == 0:
                        continue

                    if track_id not in track_faces:
                        track_faces[track_id] = []

                    track_faces[track_id].append(face_crop)

            frame_idx += 1

        scene_data.append({
            "start": start,
            "end": end,
            "track_ids": list(track_ids_in_scene)
        })

    cap.release()
    return scene_data, track_faces

In [4]:
def get_avg_embeddings(track_faces, face_app):
    track_embeddings = {}

    for track_id, crops in tqdm(track_faces.items(), desc="Analyzing faces"):
        embeddings = []
        for face in crops:
            faces = face_app.get(face)
            if faces:
                embeddings.append(faces[0].embedding)

        if embeddings:
            avg_emb = np.mean(embeddings, axis=0)
            track_embeddings[track_id] = avg_emb

    return track_embeddings

In [5]:
def cluster_track_ids(track_embeddings):
    X = list(track_embeddings.values())
    ids = list(track_embeddings.keys())

    clustering = DBSCAN(eps=0.7, min_samples=1, metric='cosine').fit(X)

    # Преобразование track_id → person_X
    track_id_to_person = {
        track_id: f"person_{label}"
        for track_id, label in zip(ids, clustering.labels_)
    }
    return track_id_to_person

In [6]:
def enrich_scenes_with_characters(scene_data, track_id_to_person):
    for scene in scene_data:
        scene['characters'] = list({
            track_id_to_person.get(tid)
            for tid in scene['track_ids']
            if tid in track_id_to_person
        })
    return scene_data

In [7]:
sentenceTransformer = SentenceTransformer('all-MiniLM-L6-v2', device="cuda")

In [8]:
def normalize_text(t):
    return re.sub(r'[^\w\s]', '', t.lower()).strip()

In [9]:
def get_text_score(a, b):
    a, b = normalize_text(a), normalize_text(b)
    if not a or not b:
        return 0.0
    a_embed = sentenceTransformer.encode(a, convert_to_tensor=True)
    b_embed = sentenceTransformer.encode(b, convert_to_tensor=True)
    return float(util.cos_sim(a_embed, b_embed)[0][0])

In [10]:
def jaccard_similarity(a, b):
    set_a, set_b = set(a), set(b)
    intersection = set_a & set_b
    union = set_a | set_b
    return len(intersection) / len(union) if union else 0

In [11]:
def group_semantic_scenes(scene_data, char_thresh=0.5, text_thresh=0.55, audio_thresh=0.02):
    grouped = []
    buffer = [scene_data[0]]

    def get_identities(scene):
        return scene.get('characters', []) or scene.get('track_ids', [])

    prev_text = scene_data[0]['transcript']
    prev_embed = sentenceTransformer.encode(normalize_text(prev_text), convert_to_tensor=True)

    for curr in scene_data[1:]:
        last = buffer[-1]
        curr_ids = get_identities(curr)
        last_ids = get_identities(last)
        char_score = jaccard_similarity(curr_ids, last_ids)

        # Нормализованный текст и эмбеддинг
        curr_text = curr['transcript']
        text_score = get_text_score(prev_text, curr_text)

        audio_diff = abs(curr['avg_rms'] - last['avg_rms'])

        if char_score >= char_thresh and text_score >= text_thresh and audio_diff <= audio_thresh:
            buffer.append(curr)
            prev_text = " ".join([prev_text, curr_text])  # копим контекст
        else:
            grouped.append({
                'start': buffer[0]['start'],
                'end': buffer[-1]['end'],
                'characters': sorted(set().union(*(get_identities(b) for b in buffer))),
                'transcript': " ".join(b['transcript'] for b in buffer),
                'avg_rms': float(np.mean([b['avg_rms'] for b in buffer]))
            })
            buffer = [curr]
            prev_text = curr['transcript']

    # Последний кусок
    if buffer:
        grouped.append({
            'start': buffer[0]['start'],
            'end': buffer[-1]['end'],
            'characters': sorted(set().union(*(get_identities(b) for b in buffer))),
            'transcript': " ".join(b['transcript'] for b in buffer),
            'avg_rms': float(np.mean([b['avg_rms'] for b in buffer]))
        })

    return grouped

In [12]:
splitter = CLIPSceneSplitter()
detector = ObjectDetector()
tracker = DeepSortTracker()

In [13]:
video_path = "test3.mp4"

In [14]:
scenes, track_faces = analyze_video(video_path, splitter, detector, tracker)

Analyzing scenes: 100%|██████████| 393/393 [00:58<00:00,  6.69it/s]


In [15]:
face_app = FaceAnalysis(name="buffalo_l", providers=["CUDAExecutionProvider"])
face_app.prepare(ctx_id=0)  # 0 — для GPU, или -1 для CPU

Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}, 'CUDAExecutionProvider': {'sdpa_kernel': '0', 'use_tf32': '1', 'fuse_conv_bias': '0', 'prefer_nhwc': '0', 'tunable_op_max_tuning_duration_ms': '0', 'enable_skip_layer_norm_strict_mode': '0', 'tunable_op_tuning_enable': '0', 'tunable_op_enable': '0', 'use_ep_level_unified_stream': '0', 'device_id': '0', 'has_user_compute_stream': '0', 'gpu_external_empty_cache': '0', 'cudnn_conv_algo_search': 'EXHAUSTIVE', 'cudnn_conv1d_pad_to_nc1d': '0', 'gpu_mem_limit': '18446744073709551615', 'gpu_external_alloc': '0', 'gpu_external_free': '0', 'arena_extend_strategy': 'kNextPowerOfTwo', 'do_copy_in_default_stream': '1', 'enable_cuda_graph': '0', 'user_compute_stream': '0', 'cudnn_conv_use_max_workspace': '1'}}
find model: /home/artniz/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'],

In [16]:
track_embeddings = get_avg_embeddings(track_faces, face_app)

Analyzing faces: 100%|██████████| 182/182 [02:58<00:00,  1.02it/s]


In [17]:
track_id_to_person = cluster_track_ids(track_embeddings)

In [18]:
scene_data = enrich_scenes_with_characters(scenes, track_id_to_person)

In [60]:
# for (start, end) in scenes:
#     print(f"start {start / 60}, end {end / 60}")
scene_data

[{'start': 0.0,
  'end': 18.996826311899483,
  'track_ids': ['1', '2'],
  'characters': ['person_0']},
 {'start': 18.996826311899483,
  'end': 22.996158167036217,
  'track_ids': ['1', '2'],
  'characters': ['person_0']},
 {'start': 22.996158167036217,
  'end': 26.99549002217295,
  'track_ids': ['1', '26', '2'],
  'characters': ['person_0']},
 {'start': 26.99549002217295,
  'end': 27.49540650406504,
  'track_ids': ['1', '26', '2'],
  'characters': ['person_0']},
 {'start': 27.49540650406504,
  'end': 38.49356910569106,
  'track_ids': ['1', '26', '28', '2'],
  'characters': ['person_0']},
 {'start': 38.49356910569106,
  'end': 47.9919822616408,
  'track_ids': ['28', '2', '38', '1', '26'],
  'characters': ['person_1', 'person_0']},
 {'start': 47.9919822616408,
  'end': 57.49039541759054,
  'track_ids': ['28', '2', '38', '1', '26'],
  'characters': ['person_1', 'person_0']},
 {'start': 57.49039541759054,
  'end': 59.49006134515891,
  'track_ids': ['1', '28', '2', '38'],
  'characters': ['p

In [19]:
def print_scenes_formatted(scenes):
    for i, scene in enumerate(scenes):
        print(f"\n🎬 Сцена {i + 1}")
        print(f"⏱ Время: {scene['start']:.2f} – {scene['end']:.2f} сек")
        print(f"🧍 Персонажи (characters): {', '.join(scene['characters'])}")
        print(f"🔊 Средняя громкость: {scene['avg_rms']:.4f}")
        print(f"🗣 Реплика:\n\"{scene['transcript'].strip()}\"")
        print("-" * 60)

In [20]:
audio_path = extract_audio_ffmpeg(video_path)

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [3]:
audio_path = 'audio.wav'

In [4]:
whisper_result = transcribe_audio_whisper(audio_path, model_size="small")
segments = whisper_result['segments']

100%|███████████████████████████████████████| 461M/461M [04:43<00:00, 1.71MiB/s]


In [5]:
segments

[{'id': 0,
  'seek': 0,
  'start': 0.0,
  'end': 6.0,
  'text': ' Итак, если потом направлен через плоскость двумя щелями в ней, и каждый из них под наблюдением, потом никогда не пойдет через обе.',
  'tokens': [50364,
   28793,
   11,
   8042,
   16873,
   36437,
   11586,
   17341,
   9283,
   1885,
   3752,
   3167,
   7196,
   5525,
   681,
   9427,
   15292,
   6293,
   740,
   23227,
   11,
   1006,
   27628,
   3943,
   14319,
   4095,
   47147,
   21297,
   11,
   16873,
   29375,
   1725,
   41207,
   1094,
   17341,
   3348,
   387,
   13,
   50664],
  'temperature': 0.0,
  'avg_logprob': -0.1430212250838043,
  'compression_ratio': 2.127725856697819,
  'no_speech_prob': 0.19534175097942352},
 {'id': 1,
  'seek': 0,
  'start': 6.0,
  'end': 8.0,
  'text': ' И если не под наблюдением, то пройдет.',
  'tokens': [50664,
   3272,
   8042,
   1725,
   4095,
   47147,
   21297,
   11,
   4572,
   1285,
   19955,
   1094,
   13,
   50764],
  'temperature': 0.0,
  'avg_logprob': -0.14

In [23]:
energy = get_audio_activity(audio_path, frame_duration=1.0)

In [66]:
energy

[np.float32(0.16264759),
 np.float32(0.150243),
 np.float32(0.13620412),
 np.float32(0.14641634),
 np.float32(0.1160831),
 np.float32(0.14108847),
 np.float32(0.116069324),
 np.float32(0.1685106),
 np.float32(0.16903909),
 np.float32(0.1641956),
 np.float32(0.12179163),
 np.float32(0.15480119),
 np.float32(0.13400128),
 np.float32(0.15214443),
 np.float32(0.13029236),
 np.float32(0.12682654),
 np.float32(0.14152427),
 np.float32(0.16185015),
 np.float32(0.12639138),
 np.float32(0.1372111),
 np.float32(0.14202033),
 np.float32(0.1447492),
 np.float32(0.108699314),
 np.float32(0.12083375),
 np.float32(0.13751379),
 np.float32(0.12966453),
 np.float32(0.11217012),
 np.float32(0.17205766),
 np.float32(0.12412887),
 np.float32(0.1772981),
 np.float32(0.16981901),
 np.float32(0.15643805),
 np.float32(0.1428202),
 np.float32(0.14184679),
 np.float32(0.15107803),
 np.float32(0.16965036),
 np.float32(0.17541796),
 np.float32(0.11583511),
 np.float32(0.16094281),
 np.float32(0.14946741),
 np.flo

In [24]:
scene_data_copy = scene_data.copy()

In [25]:
# TODO
def enrich_scenes_with_audio(scenes, segments, energy, frame_duration=1.0):
    def clip_text_to_scene(seg, start, end):
        seg_start, seg_end = seg["start"], seg["end"]
        overlap_start = max(start, seg_start)
        overlap_end = min(end, seg_end)
        overlap_dur = max(0.0, overlap_end - overlap_start)
        total_dur = seg_end - seg_start

        if overlap_dur == 0 or total_dur == 0:
            return ""

        words = seg["text"].strip().split()
        if len(words) <= 2:
            return seg["text"].strip()

        ratio = overlap_dur / total_dur
        count = max(1, int(len(words) * ratio))
        return " ".join(words[:count]) if overlap_start == seg_start else " ".join(words[-count:])

    for scene in scenes:
        start, end = scene["start"], scene["end"]

        # Текст с частичным включением
        scene_texts = [
            clip_text_to_scene(seg, start, end)
            for seg in segments
            if not (seg["end"] < start or seg["start"] > end)
        ]
        scene["transcript"] = " ".join(filter(None, scene_texts)).strip()

        # Энергия
        start_idx = max(0, int(start // frame_duration))
        end_idx = min(len(energy) - 1, int(end // frame_duration))
        scene_energy = energy[start_idx:end_idx+1] if end_idx >= start_idx else []
        scene["avg_rms"] = float(np.mean(scene_energy)) if scene_energy else 0.0

    return scenes

In [26]:
scenes_enrich = enrich_scenes_with_audio(scene_data_copy, segments, energy)

In [70]:
scenes_enrich

[{'start': 0.0,
  'end': 18.996826311899483,
  'track_ids': ['1', '2'],
  'characters': ['person_0'],
  'transcript': 'Всем привет! Я же в повар Максим Кураков. Скоро Новый год и нужно накрыть красивый вкусный стол для всей семьи. Да. В этом нам сегодня поможет на Таша Королёва. Да, всем привет, привет Максим. Рада быть сегодня с вами. Сготовка я тебе помогу и подскажу, как сделать блюдо еще вкуснее, а подачу праздничный. На',
  'avg_rms': 0.1431642472743988},
 {'start': 18.996826311899483,
  'end': 22.996158167036217,
  'track_ids': ['1', '2'],
  'characters': ['person_0'],
  'transcript': 'будет крутить к лицо, чтобы узнать для кого ищлено в семье, она будет готовить. Мы',
  'avg_rms': 0.13181427121162415},
 {'start': 22.996158167036217,
  'end': 26.99549002217295,
  'track_ids': ['1', '26', '2'],
  'characters': ['person_0'],
  'transcript': 'приготовим всего 4 блюдо. Не пропустите секрет о готовке на',
  'avg_rms': 0.12177629768848419},
 {'start': 26.99549002217295,
  'end': 27.495

In [27]:
scenes_enrich_copy = scenes_enrich.copy()

In [72]:
scenes_enrich_copy

[{'start': 0.0,
  'end': 18.996826311899483,
  'track_ids': ['1', '2'],
  'characters': ['person_0'],
  'transcript': 'Всем привет! Я же в повар Максим Кураков. Скоро Новый год и нужно накрыть красивый вкусный стол для всей семьи. Да. В этом нам сегодня поможет на Таша Королёва. Да, всем привет, привет Максим. Рада быть сегодня с вами. Сготовка я тебе помогу и подскажу, как сделать блюдо еще вкуснее, а подачу праздничный. На',
  'avg_rms': 0.1431642472743988},
 {'start': 18.996826311899483,
  'end': 22.996158167036217,
  'track_ids': ['1', '2'],
  'characters': ['person_0'],
  'transcript': 'будет крутить к лицо, чтобы узнать для кого ищлено в семье, она будет готовить. Мы',
  'avg_rms': 0.13181427121162415},
 {'start': 22.996158167036217,
  'end': 26.99549002217295,
  'track_ids': ['1', '26', '2'],
  'characters': ['person_0'],
  'transcript': 'приготовим всего 4 блюдо. Не пропустите секрет о готовке на',
  'avg_rms': 0.12177629768848419},
 {'start': 26.99549002217295,
  'end': 27.495

In [28]:
scenes_grouped = group_semantic_scenes(scenes_enrich_copy)

In [29]:
def clean_and_merge_short_scenes(scenes, min_duration=2.0, min_words=3):
    cleaned = []
    buffer = None

    def merge_scene(a, b):
        return {
            "start": a["start"],
            "end": b["end"],
            "characters": sorted(set(a.get("characters", []) + b.get("characters", []))),
            "transcript": " ".join([a["transcript"], b["transcript"]]).strip(),
            "avg_rms": float(np.mean([a["avg_rms"], b["avg_rms"]]))
        }

    for scene in scenes:
        duration = scene["end"] - scene["start"]
        word_count = len(scene["transcript"].strip().split())

        # Пропуск пустых или бессмысленных сцен
        if word_count < min_words or duration < 0.5:
            if cleaned:
                cleaned[-1] = merge_scene(cleaned[-1], scene)
            continue

        if duration < min_duration:
            if cleaned:
                cleaned[-1] = merge_scene(cleaned[-1], scene)
            else:
                buffer = scene
        else:
            cleaned.append(scene)

    # Добавляем буфер, если остался
    if buffer:
        if cleaned:
            cleaned[-1] = merge_scene(cleaned[-1], buffer)
        else:
            cleaned.append(buffer)

    return cleaned

In [30]:
scenes_grouped = clean_and_merge_short_scenes(scenes_grouped, min_duration=2.0, min_words=3)

In [76]:
scenes_grouped

[{'start': 0.0,
  'end': 18.996826311899483,
  'characters': ['person_0'],
  'transcript': 'Всем привет! Я же в повар Максим Кураков. Скоро Новый год и нужно накрыть красивый вкусный стол для всей семьи. Да. В этом нам сегодня поможет на Таша Королёва. Да, всем привет, привет Максим. Рада быть сегодня с вами. Сготовка я тебе помогу и подскажу, как сделать блюдо еще вкуснее, а подачу праздничный. На',
  'avg_rms': 0.1431642472743988},
 {'start': 18.996826311899483,
  'end': 27.49540650406504,
  'characters': ['person_0'],
  'transcript': 'будет крутить к лицо, чтобы узнать для кого ищлено в семье, она будет готовить. Мы приготовим всего 4 блюдо. Не пропустите секрет о готовке на Таша. Это',
  'avg_rms': 0.13445458933711052},
 {'start': 27.49540650406504,
  'end': 38.49356910569106,
  'characters': ['person_0'],
  'transcript': 'уже на 3-й выпуск для Сергей и мамы уже приготовили блюдо в предыдущих программах, кто еще не видел, скорее посмотрите. А чтобы вы близкого порадуем в этом выпус

In [31]:
print_scenes_formatted(scenes_grouped)


🎬 Сцена 1
⏱ Время: 0.63 – 4.38 сек
🧍 Персонажи (characters): 
🔊 Средняя громкость: 0.1096
🗣 Реплика:
"потом никогда не пойдет через обе и если не под наблюдением то пройдет."
------------------------------------------------------------

🎬 Сцена 2
⏱ Время: 4.38 – 24.40 сек
🧍 Персонажи (characters): person_0
🔊 Средняя громкость: 0.0478
🗣 Реплика:
"то пройдет. то пройдет. то пройдет. если не под наблюдением то пройдет. Но если потом не пойдет с хвостовищили. потом не пойдет с хвостовищили. пойдет с хвостовищили. Согласен, а в чем скорой идее для принтерна футболки. скорой идее для принтерна футболки. футболки. Пошла"
------------------------------------------------------------

🎬 Сцена 3
⏱ Время: 24.40 – 38.79 сек
🧍 Персонажи (characters): person_0
🔊 Средняя громкость: 0.0774
🗣 Реплика:
"прощение секунду. Так, по-корезантале этой гейске 8 поверхекали на боков 26 по-гризантали ПВС, 14 по-ритикалипали. Суберитесь здесь. Сипа,"
------------------------------------------------------------

🎬

In [32]:
output_dir = "output_scenes"

save_scenes_ffmpeg(video_path, scenes_grouped, output_dir)

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab